In [2]:
args <- commandArgs(T) 

In [3]:
library(here)
library(rjson)
library(Matrix)
library(tidyverse)
library(dplyr)
library(DropletUtils) 

library(Seurat)
library(Signac)
library(SeuratDisk)

library(rhdf5)
library(anndata)

# convert H5Seurat

## Multiome

In [4]:
input_path_10k <- "/home/wsg/BM/data/10x_PBMC/RawData/PBMC_10k"
input_path_3k <- "/home/wsg/BM/data/10x_PBMC/RawData/PBMC_3k"
output_path <- "/home/wsg/BM/data/10x_PBMC/RNA+ATAC/RawData"

In [5]:
pbmc_10k <- Read10X_h5(file = here(input_path_10k, "pbmc_unsorted_10k_filtered_feature_bc_matrix.h5"))

Genome matrix has multiple modalities, returning a list of matrices for this genome



In [6]:
pbmc_3k <- Read10X_h5(file = here(input_path_3k, "pbmc_unsorted_3k_filtered_feature_bc_matrix.h5"))

Genome matrix has multiple modalities, returning a list of matrices for this genome



## metadata

In [7]:
metadata_10k <- data.frame(barcode = paste0(colnames(pbmc_10k$`Gene Expression`), "-10k"),
                           batch = "pbmc_10k")
metadata_3k <- data.frame(barcode = paste0(colnames(pbmc_3k$`Gene Expression`), "-3k"),
                          batch = "pbmc_3k")
metadata <- rbind(metadata_10k, metadata_3k)
rownames(metadata) <- metadata$barcode
# metadata

In [8]:
table(metadata$batch)
write_csv(metadata, here(output_path, "metadata.csv"))


pbmc_10k  pbmc_3k 
   12012     3009 

## RNA

In [9]:
RNA_counts_10k <- pbmc_10k$`Gene Expression`
RNA_counts_3k <- pbmc_3k$`Gene Expression`

In [10]:
colnames(RNA_counts_10k) <- paste0(colnames(RNA_counts_10k), "-10k")
colnames(RNA_counts_3k) <- paste0(colnames(RNA_counts_3k), "-3k")

In [11]:
if(all(rownames(RNA_counts_10k) == rownames(RNA_counts_3k))){
    RNA_counts <- cbind(RNA_counts_10k, RNA_counts_3k)
}

In [12]:
RNA_counts

  [[ suppressing 32 column names ‘AAACAGCCAAAGCCTC-1-10k’, ‘AAACAGCCAGAATGAC-1-10k’, ‘AAACAGCCAGCTACGT-1-10k’ ... ]]

  [[ suppressing 32 column names ‘AAACAGCCAAAGCCTC-1-10k’, ‘AAACAGCCAGAATGAC-1-10k’, ‘AAACAGCCAGCTACGT-1-10k’ ... ]]

  [[ suppressing 32 column names ‘AAACAGCCAAAGCCTC-1-10k’, ‘AAACAGCCAGAATGAC-1-10k’, ‘AAACAGCCAGCTACGT-1-10k’ ... ]]



36601 x 15021 sparse Matrix of class "dgCMatrix"
                                                                                            
MIR1302-2HG   .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
FAM138A       .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
OR4F5         .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AL627309.1    .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  1  . .  .  . .  .
AL627309.3    .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AL627309.2    .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AL627309.5    .  .  .  . . . . . . .  1 . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AL627309.4    .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AP006222.2    .  .  .  . . . . . . .  . . . . . .  .  . . .  .  .  .  . .  .  . .  .  . .  .
AL732372.1    .  .  .

In [13]:
RNA_subset_counts <- RNA_counts

In [14]:
process = "raw"

# save raw rna to mtx
data_path <- here(output_path, "PBMC-multiome-raw-RNA-counts.mtx")
write10xCounts(x = RNA_subset_counts, path = data_path, version = "3")

# save raw rna to rds
saveRDS(RNA_subset_counts, 
        file = here(output_path, "PBMC-multiome-raw-RNA-counts.rds"))

# Create Seurat Object
RNA_subset <- CreateSeuratObject(counts = RNA_subset_counts, meta.data = metadata)

# save Seurat to h5Seurat
SaveH5Seurat(RNA_subset, overwrite = TRUE, 
             filename = here(output_path, "PBMC-multiome-raw-RNA-counts.h5Seurat"))

# Convert h5Seurat to h5ad
setwd(output_path)
Convert(here(output_path, "PBMC-multiome-raw-RNA-counts.h5Seurat"), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Validating h5Seurat file

Adding data from RNA as X

Adding counts from RNA as raw

Transfering meta.data to obs



## ATAC

In [108]:
ATAC_counts_10k <- pbmc_10k$Peaks
ATAC_counts_3k <- pbmc_3k$Peaks

In [109]:
colnames(ATAC_counts_10k) <- paste0(colnames(ATAC_counts_10k), "-10k")
colnames(ATAC_counts_3k) <- paste0(colnames(ATAC_counts_3k), "-3k")

In [110]:
length(intersect(rownames(ATAC_counts_10k), rownames(ATAC_counts_3k)))

[1] 70

In [111]:
# remap the peaks
library(rtracklayer)

merged_peaks_path <- paste0(output_path, "/merged.bed")
merged_peaks <- import(merged_peaks_path, format = "BED")
merged_peaks

GRanges object with 115980 ranges and 0 metadata columns:
             seqnames          ranges strand
                <Rle>       <IRanges>  <Rle>
       [1]       chr1      9783-10676      *
       [2]       chr1   180548-181446      *
       [3]       chr1   191122-192066      *
       [4]       chr1   267554-268458      *
       [5]       chr1   270882-271782      *
       ...        ...             ...    ...
  [115976] KI270727.1     52088-52969      *
  [115977] KI270728.1   232229-233172      *
  [115978] KI270728.1 1791271-1791983      *
  [115979] KI270728.1 1792101-1792584      *
  [115980] KI270734.1   133496-134405      *
  -------
  seqinfo: 37 sequences from an unspecified genome; no seqlengths

In [112]:
library(GenomicRanges)
library(Matrix)

# 将两个原始的peaks矩阵的行名转换为GRanges对象
gr_10k <- GRanges(rownames(ATAC_counts_10k))
gr_3k <- GRanges(rownames(ATAC_counts_3k))

# 计算每个原始peak与合并后peaks的重叠情况
ov_10k <- findOverlaps(gr_10k, merged_peaks)
ov_3k <- findOverlaps(gr_3k, merged_peaks)

In [113]:
merged_peaks_name <- paste0(merged_peaks@seqnames, "-", start(merged_peaks), "-", end(merged_peaks))

In [118]:
head(ov_10k)
head(queryHits(ov_10k))
head(subjectHits(ov_10k))

rownames(ATAC_counts_10k)[head(queryHits(ov_10k))]
merged_peaks_name[head(subjectHits(ov_10k))]

Hits object with 6 hits and 0 metadata columns:
      queryHits subjectHits
      <integer>   <integer>
  [1]         1           1
  [2]         2           2
  [3]         3           3
  [4]         4           4
  [5]         5           5
  [6]         6           7
  -------
  queryLength: 111857 / subjectLength: 115980

[1] 1 2 3 4 5 6

[1] 1 2 3 4 5 7

[1] "chr1:9790-10676"    "chr1:180654-181318" "chr1:191155-192066"
[4] "chr1:267573-268458" "chr1:270881-271760" "chr1:585751-586643"

[1] "chr1-9783-10676"    "chr1-180548-181446" "chr1-191122-192066"
[4] "chr1-267554-268458" "chr1-270882-271782" "chr1-585752-586649"

In [123]:
ATAC_counts_10k_rename <- ATAC_counts_10k
rownames(ATAC_counts_10k_rename) <- merged_peaks_name[subjectHits(ov_10k)]

In [124]:
ATAC_counts_3k_rename <- ATAC_counts_3k
rownames(ATAC_counts_3k_rename) <- merged_peaks_name[subjectHits(ov_3k)]

In [125]:
length(intersect(rownames(ATAC_counts_10k_rename), rownames(ATAC_counts_3k_rename)))

[1] 76823

In [144]:
ATAC_counts <- SeuratObject::RowMergeSparseMatrices(ATAC_counts_10k_rename, ATAC_counts_3k_rename)

In [146]:
ATAC_subset_counts <- ATAC_counts

In [147]:
# save raw ATAC to mtx
data_path <- here(output_path, "PBMC-multiome-raw-ATAC-peaks.mtx")
write10xCounts(x = ATAC_subset_counts, path = data_path, version = "3")

# save raw ATAC to rds
saveRDS(ATAC_subset_counts, 
        file = here(output_path, "PBMC-multiome-raw-ATAC-peaks.rds"))

# Create Seurat Object
ATAC_subset <- CreateSeuratObject(counts = ATAC_subset_counts, meta.data = metadata)

# save Seurat to h5Seurat
SaveH5Seurat(ATAC_subset, overwrite = TRUE, 
             filename = here(output_path, "PBMC-multiome-raw-ATAC-peaks.h5Seurat"))

# Convert h5Seurat to h5ad
setwd(output_path)
Convert(here(output_path, "PBMC-multiome-raw-ATAC-peaks.h5Seurat"), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Validating h5Seurat file

Adding data from RNA as X

Adding counts from RNA as raw

Transfering meta.data to obs

